In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
df=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas1=df['cell_area'].transpose('lat','lon')
areas1=areas1/float(1e6)

lons_east=[x for x in areas1.lon.values if x>=90.]
lons_west=[x for x in areas1.lon.values if x<-90.]

areas_east=areas1.sel(lon=lons_east)
areas_west=areas1.sel(lon=lons_west)

areas_east['lon']=[x-360. for x in lons_east]
areas_west['lon']=[x+360. for x in lons_west]

areas=xr.concat([areas_east,areas1,areas_west],dim='lon')

In [ ]:
print(areas.max().values)
print(areas.lon.values[0:10])
print(areas.lat.values[0:10])
print(areas)

In [ ]:
ivt_threshold=300.
count_fig=0
yrs=np.arange(1940,1941,1)
count_stop=0
count_cont=0


ds=xr.open_dataset(dir2+'era5.ar.labels.model.1982.nc')
l1=ds['ar_labeled']

for l in range(len(yrs)):
    year=yrs[l]
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt_mag_current1=ds['ivt'][0,:,:]
    
    lons_east=[x for x in ds.lon.values if x>=90.]
    lons_west=[x for x in ds.lon.values if x<-90.]
    
    ivt_mag_current_east=ivt_mag_current1.sel(lon=lons_east)
    ivt_mag_current_west=ivt_mag_current1.sel(lon=lons_west)
    
    ivt_mag_current_east['lon']=[x-360. for x in lons_east]
    ivt_mag_current_west['lon']=[x+360. for x in lons_west]
    
    ivt_mag_current=xr.concat([ivt_mag_current_east,ivt_mag_current1,ivt_mag_current_west],dim='lon')
    
    lons_extended=ivt_mag_current.lon.values.tolist()

    lats=ds.lat.values.tolist()
    lons=lons_extended
    dates_unique=ds.time.values
    
    lat_array=xr.zeros_like(ivt_mag_current[:,:])
    lon_array=xr.zeros_like(ivt_mag_current[:,:])
    for i in range(ivt_mag_current.lon.size):
        lat_array[:,i]=ivt_mag_current.lat.values
    for i in range(ivt_mag_current.lat.size):
        lon_array[i,:]=ivt_mag_current.lon.values
        
    lon_array_list=lon_array.values.ravel()
    lat_array_list=lat_array.values.ravel()
    zipped_latlon=list(zip(lat_array_list,lon_array_list))

    #indices=np.random.choice(np.arange(1464),150)
    indices=[552]#82
    #indices=[1460-7]#81
    #indices=[828]#80
    #indices=[1108-3-4]
    
    #for d in range(len(dates_unique)):#2161
    #for d in indices:
    for d in range(10):
            
        date=pd.to_datetime(dates_unique[d])
        print(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ivt_mag_current1=ds['ivt'].sel(time=date)
        
        lons_east=[x for x in ds.lon.values if x>=90.]
        lons_west=[x for x in ds.lon.values if x<-90.]

        ivt_mag_current_east=ivt_mag_current1.sel(lon=lons_east)
        ivt_mag_current_west=ivt_mag_current1.sel(lon=lons_west)

        ivt_mag_current_east['lon']=[x-360. for x in lons_east]
        ivt_mag_current_west['lon']=[x+360. for x in lons_west]

        ivt_mag_current=xr.concat([ivt_mag_current_east,ivt_mag_current1,ivt_mag_current_west],dim='lon')
        
        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        structure = np.ones((3, 3))          
        labeled,ncomponents=label(ivt_ones,structure)
        labeled_xr=xr.zeros_like(ivt_extreme)
        labeled_xr[:,:]=labeled
        labeled_xr_flat=labeled_xr.values.flatten()
        labeled_xr_original=labeled_xr.copy()

        ar_timestep=xr.zeros_like(labeled_xr)
        
        for i in range(ncomponents+1):
            distance_max=0
            select_component=labeled_xr.where(labeled_xr==i)
            ivt_component=ivt_ones.where(labeled_xr==i)
            ivt_sum=ivt_component.sum(skipna=True).values
            
            if 1==0:
                test=select_component/select_component
                test=test.fillna(0)

                fig=plt.figure(figsize=(22,9))
                ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                cax1=ax.contourf(test.lon,test.lat,test.where(test>0),levels=[0,1],colors='b',transform=ccrs.PlateCarree(),alpha=0.75)
                cbar=plt.colorbar(cax1,pad=0,fraction=0.046)
                cbar.ax.tick_params(labelsize=20) 
                cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=21,labelpad=5)

                g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')

                ax.coastlines(resolution='10m')
                #ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
                #ax.add_feature(cfeature.LAKES.with_scale('50m'))
                countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
                ax.add_feature(countries)
                g1.xlabel_style={'size':20,'color':'k'}
                g1.ylabel_style={'size':20,'color':'k'}
                g1.xformatter=LONGITUDE_FORMATTER
                g1.yformatter=LATITUDE_FORMATTER
                #g1.xlocator = mticker.FixedLocator([119,120,121,122,123])
                #g1.ylocator = mticker.FixedLocator([21,22,23,24,25,26])

                g1.top_labels=False
                g1.right_labels=False
                ax.set_xlim(-180,180)
                ax.set_ylim(-75,75)

                ax.set_title('AR detection: '+date.strftime('%Y.%m.%d.%H'),fontsize=27,pad=5)
                plt.show()


            
            if ivt_sum>=60.:
                lats_where=lat_array.where(labeled_xr==i)
                lons_where=lon_array.where(labeled_xr==i)
                min_lat=lats_where.min(skipna=True).values
                max_lat=lats_where.max(skipna=True).values
                min_lon=lons_where.min(skipna=True).values
                max_lon=lons_where.max(skipna=True).values
                distance_hyp=geopy.distance.geodesic((min_lat,min_lon),(max_lat,max_lon)).km
                
                if distance_hyp>=2000:#filter out features that could not possibly be long enough
                    choose_lats=lat_array.where(labeled_xr==i)
                    choose_lons=lon_array.where(labeled_xr==i)
                    
                    lats_flat=choose_lats.values.ravel()
                    lons_flat=choose_lons.values.ravel()
                  
                    zipped_ar=list(zip(lats_flat,lons_flat))
                    zipped_ar=[x for x in zipped_ar if x[0]>=-90]

                    hull=ConvexHull(zipped_ar)

                    hull_points=[]
                    for j in range(len(hull.vertices)):
                        hull_points.append(zipped_ar[hull.vertices[j]])

                    distance_list=[]
                    for j in range(len(hull_points)):
                        for k in range(len(hull_points)):
                            point_1=hull_points[j]
                            point_2=hull_points[k]
                            distance=geopy.distance.distance(point_1,point_2).km
                            distance_list.append(distance)

                    length=max(distance_list)
                    
                    area_sum=areas.where(labeled_xr==i).sum(dim=('lat','lon'),skipna=True).values.tolist()

                    width=area_sum/length
                    
                    #Now apply tropical condition
                    
                    component_tropical=select_component.where(lat_array<=20)
                    component_tropical=component_tropical.where(lat_array>=-20)
                    component_tropical_filled=component_tropical.fillna(0)
                    component_nontropical=select_component.where(component_tropical_filled==0)

                    area_tropical=areas.where(component_tropical==i).sum(dim=('lat','lon'),skipna=True).values
                    area_nontropical=areas.where(component_nontropical==i).sum(dim=('lat','lon'),skipna=True).values
                    
                    CONTINUE='yes'
                    
                    if 1==1:
                        if area_tropical/area_nontropical<1./2.5 and length/width>=2.5 and length>=2000.:
                            
                            final_region=labeled_xr.where(labeled_xr==i).fillna(0)
                            final_region=final_region/final_region
                            final_region=final_region.fillna(0)
                            ar_timestep=ar_timestep+final_region
                                
                            count_stop=count_stop+1
                            CONTINUE='no'
                        
                        elif length>=2000.:
                            CONTINUE='yes'
                        else:
                            CONTINUE='no'
                            
                        if area_nontropical==0:
                            CONTINUE='no'
                            
                        count_extra=0
                        
                        higher_threshold=ivt_threshold+50.
                        
                        while higher_threshold<=550. and CONTINUE=='yes':
                            CONTINUE='no'
                            count_extra=count_extra+1
                            ivt_higher_threshold=ivt_mag_current.where(ivt_mag_current>=higher_threshold)
                            region_specific=ivt_higher_threshold.where(labeled_xr==i)
                            binary=region_specific/region_specific
                            binary=binary.fillna(0)
                            
                            labeled_new=xr.zeros_like(binary)
                            labeled_new_values,n=label(binary,structure)
                            labeled_new[:,:]=labeled_new_values
                            labeled_new_flat=labeled_new.values.flatten()
                            for k1 in range(n+1):
                                distance_max=0
                                select_component=labeled_new.where(labeled_new==k1)
                                
                                ivt_sum=binary.where(labeled_new==k1).sum(skipna=True).values
                                
                                if ivt_sum>=60.:
                                    
                                    lats_where=lat_array.where(labeled_new==k1)
                                    lons_where=lon_array.where(labeled_new==k1)
                                    min_lat=lats_where.min(skipna=True).values
                                    max_lat=lats_where.max(skipna=True).values
                                    min_lon=lons_where.min(skipna=True).values
                                    max_lon=lons_where.max(skipna=True).values
                                    distance_hyp=geopy.distance.geodesic((min_lat,min_lon),(max_lat,max_lon)).km
                                    
                                    if distance_hyp>=2000:#filter out features that could not possibly be long enough
                                    
                                        area_sum=areas.where(labeled_new==k1).sum(dim=('lat','lon'),skipna=True).values.tolist()

                                        choose_lats=lat_array.where(labeled_new==k1)
                                        choose_lons=lon_array.where(labeled_new==k1)

                                        lats_flat=choose_lats.values.ravel()
                                        lons_flat=choose_lons.values.ravel()
                                        
                                        zipped_ar=list(zip(lats_flat,lons_flat))
                                        zipped_ar=[x for x in zipped_ar if x[0]>=-90]

                                        hull=ConvexHull(zipped_ar)

                                        hull_points=[]
                                        for j2 in range(len(hull.vertices)):
                                            hull_points.append(zipped_ar[hull.vertices[j2]])

                                        distance_list=[]
                                        for j2 in range(len(hull_points)):
                                            for k2 in range(len(hull_points)):
                                                point_1=hull_points[j2]
                                                point_2=hull_points[k2]
                                                distance=geopy.distance.distance(point_1,point_2).km
                                                distance_list.append(distance)

                                        length=max(distance_list)
                                        width=area_sum/length
                                        
                                        component_tropical=select_component.where(lat_array<=20)
                                        component_tropical=component_tropical.where(lat_array>=-20)
                                        component_tropical_filled=component_tropical.fillna(0)
                                        component_nontropical=select_component.where(component_tropical_filled==0)

                                        area_tropical=areas.where(component_tropical==k1).sum(dim=('lat','lon'),skipna=True).values
                                        area_nontropical=areas.where(component_nontropical==k1).sum(dim=('lat','lon'),skipna=True).values
                                        
                                        if area_tropical/area_nontropical<1./2.5 and length/width>=2.5 and length>=2000.:

                                            label_ar_term=labeled_new.where(labeled_new==k1)
                                            label_ar_term=label_ar_term/label_ar_term
                                            label_ar_term=label_ar_term.fillna(0)
                                    
                                            #c5=ax.contour(label_ar_term.lon,label_ar_term.lat,label_ar_term,[0,1],linewidths=1.5,colors='r',transform=ccrs.PlateCarree())         

                                            count_cont=count_cont+1
                                            print('CONT')
                                            
                                            
                                            

                                            structure_dilation=np.zeros((3,3))
                                            structure_dilation[1,2]=1
                                            structure_dilation[0,1]=1
                                            structure_dilation[1,0]=1
                                            structure_dilation[2,1]=1
                                            structure_dilation[1,1]=1
                                            region_dilate=binary_dilation(label_ar_term,structure_dilation,iterations=3+2*(count_extra-1))
                                            
                                            final_region=ivt_ones.where(region_dilate==1)
                                            final_region=final_region.where(labeled_xr_original==i).fillna(0)
                                            
                                            
                                            area_sum_mod=areas.where(final_region==1).sum(dim=('lat','lon'),skipna=True).values

                                            region_north=labeled_xr.where(labeled_xr==i).where(lat_array>=45.).fillna(0)
                                            region_south=labeled_xr.where(labeled_xr==i).where(lat_array<=-45.).fillna(0)
                                            region_polar=region_north+region_south
                                            
                                            final_region=final_region+region_polar
                                            final_region=final_region/final_region
                                            final_region=final_region.fillna(0)
                                            
                                            ar_timestep=ar_timestep+final_region
                                                
                                            ######remove this portion from the original by modifying labeled_xr
                                            region_subset=select_component/select_component
                                            region_subset=region_subset.fillna(0)
                                            region_subset=region_subset.where(region_subset==0)
                                            labeled_xr=labeled_xr+region_subset
                                            ######
                                            
                                            if 1==0:
                                                lons_center=[x for x in ivt_mag_current.lon.values if -180<=x<180]
                                            
                                                test=ivt_mag_current#.sel(lon=lons_center)
                                                test=test.where(test>0)
                                                test=test.values

                                                l1_sel=l1.sel(time=date)

                                                fig=plt.figure(figsize=(22,9))
                                                ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree(central_longitude=180))
                                                cax1=ax.contourf(ivt_mag_current.lon.values,ivt_mag_current.lat.values,test,levels=np.arange(250,1350,100),cmap=plt.cm.Greys,extend='max',transform=ccrs.PlateCarree(),alpha=0.5,label='Identified extreme IVT')


                                                test1=final_region
                                                field_final=xr.zeros_like(final_region)
                                                labeled_xr=xr.zeros_like(final_region)
                                                labeled,n=label(test1,structure)
                                                labeled_xr[:,:]=labeled
                                                for x1 in range(1,n+1):
                                                    select=labeled_xr.where(labeled_xr==x1)
                                                    select=select/select
                                                    select=select.fillna(0)
                                                    if select.sum().values>=100:
                                                        field_final=field_final+select

                                                cax2=ax.pcolormesh(l1_sel.lon,l1_sel.lat,l1_sel.where(l1_sel>0),vmin=0,vmax=l1_sel.max().values,cmap=plt.cm.jet,transform=ccrs.PlateCarree(),alpha=0.5,label='Identified AR')
                                                mpl.rcParams['hatch.linewidth']=0.01

                                                ivt_300=ivt_mag_current#.sel(lon=lons_center)
                                                ivt_300=ivt_300.where(ivt_300>=300.)
                                                ivt_300_ones=ivt_300/ivt_300
                                                ivt_300_ones=ivt_300_ones.fillna(0)


                                                c=ax.contour(ivt_300.lon,ivt_300.lat,ivt_300_ones,[0,1],colors='k',linewidths=4,transform=ccrs.PlateCarree())         
                                                g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=2.5,color='gray',alpha=0.0,linestyle='--')


                                                test=label_ar_term/label_ar_term
                                                test=test.fillna(0)
                                                c=ax.contour(test.lon,test.lat,test,[0,1],colors='magenta',linewidths=7,transform=ccrs.PlateCarree())         


                                                ax.coastlines(resolution='10m')
                                                #ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
                                                #ax.add_feature(cfeature.LAKES.with_scale('50m'))
                                                countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
                                                ax.add_feature(countries)
                                                g1.xlabel_style={'size':25,'color':'k'}
                                                g1.ylabel_style={'size':25,'color':'k'}
                                                g1.xformatter=LONGITUDE_FORMATTER
                                                g1.yformatter=LATITUDE_FORMATTER
                                                #g1.xlocator = mticker.FixedLocator([119,120,121,122,123])
                                                #g1.ylocator = mticker.FixedLocator([21,22,23,24,25,26])

                                                g1.top_labels=False
                                                g1.right_labels=False

                                                print((min_lon,max_lon))
                                                #if 180 in lons_where.values.flatten().tolist():
                                                #if min_lon<-180 and max_lon>-180 or min_lon<180 and max_lon>180:
                                                #    ax.set_xlim(-180,180)
                                                #    ax.set_ylim(-75,75)
                                                #else:
                                                #    ax.set_xlim(max(min_lon-20,-180),min(max_lon+20,180))
                                                #    ax.set_ylim(max(min_lat-20,-90),min(max_lat+20,90))

                                                #ax.set_xlim(-180,180)
                                                #ax.set_ylim(-80,80)

                                                ax.set_extent([100,-150,0,45])
                                                #ax.set_extent([-165,-65,-20,-65])
                                                #ax.set_extent([-105,-0.1,10,52.5])
                                                #ax.set_extent([-95,25,10,55])

                                                ax.set_title('ERA5 fixed AR detection: '+date.strftime('%Y.%m.%d.%H'),fontsize=36,pad=5)
                                                plt.show()
                                                #sys.exit()
                                                count_fig=count_fig+1
                                                #fig.savefig(dir_data+'method_testing_v8_'+str(d)+'.png')
                                                fig.savefig(dir_data+'method_testing_single_1.png')
                                                sys.exit()
                                                if count_fig==20:
                                                    sys.exit()
                                            
                                        elif length>=2000.:
                                            CONTINUE='yes'
                                            

                            higher_threshold=higher_threshold+50.
        
        lons_center=[x for x in ar_timestep.lon.values if -180<=x<180]#90-degree barrier on each side
        ar_timestep=ar_timestep.sel(lon=lons_center)
        ar_timestep=ar_timestep/ar_timestep#can have overlap from binary dilation
        ar_timestep=ar_timestep.fillna(0)

        ivt_timestep=ivt_mag_current.sel(lon=lons_center)
        #if d==indices[0]:
        if d==0:
            ar_points=ar_timestep
        else:
            ar_points=xr.concat([ar_points,ar_timestep],dim='time')
        
        if 1==0:
            test=ivt_timestep.where(ivt_timestep>0)
            test=test.values

            fig=plt.figure(figsize=(22,9))
            ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
            cax1=ax.contourf(ivt_timestep.lon.values,ivt_timestep.lat.values,test,levels=np.arange(250,1050,100),cmap=plt.cm.Greys,transform=ccrs.PlateCarree(),alpha=0.3,label='Identified extreme IVT',zorder=20)
            cbar=plt.colorbar(cax1,pad=0,fraction=0.046)
            cbar.ax.tick_params(labelsize=20) 
            cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=21,labelpad=5)

            cax2=ax.pcolormesh(ar_points.lon,ar_points.lat,ar_timestep.where(ar_timestep>0),vmin=0,vmax=1,cmap=plt.cm.Blues,transform=ccrs.PlateCarree(),alpha=0.5,label='Identified AR')
            mpl.rcParams['hatch.linewidth']=0.01

            ivt_300=ivt_timestep.where(ivt_timestep>=300.)
            ivt_300_ones=ivt_300/ivt_300
            ivt_300_ones=ivt_300_ones.fillna(0)

            c=ax.contour(ivt_300.lon,ivt_300.lat,ivt_300_ones,[0,1],linewidths=1.5,transform=ccrs.PlateCarree())         
            g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')

            ax.coastlines(resolution='10m')
            #ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            #ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            g1.xlabel_style={'size':20,'color':'k'}
            g1.ylabel_style={'size':20,'color':'k'}
            g1.xformatter=LONGITUDE_FORMATTER
            g1.yformatter=LATITUDE_FORMATTER
            #g1.xlocator = mticker.FixedLocator([119,120,121,122,123])
            #g1.ylocator = mticker.FixedLocator([21,22,23,24,25,26])

            g1.top_labels=False
            g1.right_labels=False
            #ax.set_xlim(-180,180)
            #ax.set_ylim(-75,75)

            ax.set_title('AR detection: '+date.strftime('%Y.%m.%d.%H'),fontsize=27,pad=5)
            plt.show()
            sys.exit()
            #fig.savefig(dir_data+'ar_detection_example_'+str(d)+'.png')
            #if d==10:
            #    sys.exit()
            #fig.savefig(dir1+'ar_test_new_'+str(year)+'_'+str(d)+'.png')
        
    #print(count_normal)
    #print(count_stop)
    #sys.exit()
    if 1==1:
        
        dk=xr.Dataset()

        dk['ar']=(('time','lat','lon'),ar_points.values)

        dk.coords['time']=ar_points.time
        dk.coords['lat']=ar_points.lat
        dk.coords['lon']=ar_points.lon
        
        dk['ar'].attrs["description"]='1 = AR, 0 = no AR'
        dk['lat'].attrs["units"]='degrees_north'
        dk['lon'].attrs["units"]='degrees_east'
        
        try:
            os.remove(dir2+'era_5_ar_detection_test_'+str(year)+'.nc')
        except OSError:
            pass
        dk.to_netcdf(dir2+'era_5_ar_detection_test_'+str(year)+'.nc',mode='w',format='NETCDF4')

In [ ]:
print(test)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test.where(test>0),vmin=0,vmax=1,cmap=plt.cm.Reds)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
print(test.max().values)

In [ ]:
ds=xr.open_dataset(dir2+'era5.ar.labels.model.1981.nc')
test=ds['ar_labeled'].sel(time=dt.datetime(1981,10,4,12))

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=test.max().values,cmap=plt.cm.RdYlGn_r)
ax.coastlines(resolution='10m')
plt.show()